# `Sign Language Detetcion using edge detection then CNN` 🪟

## import libraries 📚

In [ ]:
#%pip install -U scikit-learn

In [2]:
#imports
import os
import cv2
#import imghdr
#import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import load_model
from matplotlib import pyplot as plt
from keras.applications import ResNet50
from sklearn.preprocessing import normalize
from tensorflow.keras.preprocessing import image
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
#----------------
import keras
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D, Dense,Flatten, Dropout
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.optimizers import SGD

## Attribute initialization 🧚

In [3]:
# Attributes Initialization
labels_dir = 'Dataset\Annotation'
classes_dir= 'Dataset\Signs_grayscale'
image_exts = ['jpeg','jpg', 'bmp', 'png','JPG']                               #putting all the wanted image extensions
class_names = ['drink', 'eat', 'hello','help','love','morning','okay', 'please','stop','thankyou']
image_size = (224, 224)
batch_size = 32

## Use GPU 💻

In [5]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
tf.config.list_physical_devices('GPU')

[]

# Image Preprocessing🖼️

In [9]:
imageSize = (224, 224)
batchSize = 32

## edge detetction on each image 🍃

In [10]:

# import cv2
# import os

# # Set the root directory where the 10 folders are located
# rootdir = 'Dataset\Signs_grayscale'

# # Set the lower and upper threshold values for the Canny edge detection
# t_lower = 100
# t_upper = 200

# # Loop through the subdirectories of the root directory
# for subdir, dirs, files in os.walk(rootdir):
#     # Loop through the files in the current subdirectory
#     for file in files:
#         # Check if the file is an image (you can add more image file extensions if needed)
#         if file.endswith(('.jpg', '.jpeg', '.png')):
#             # Construct the full path to the image file
#             filepath = os.path.join(subdir, file)
            
#             # Load the image using OpenCV
#             img = cv2.imread(filepath)
            
#             # Apply Canny edge detection on the image
#             edges = cv2.Canny(img, t_lower, t_upper)
            
#             # Save the resulting image (you can change the output directory and filename as needed)
#             cv2.imwrite(filepath, edges)


## Split train and test 🪓

In [13]:

gen_data_train = tf.keras.utils.image_dataset_from_directory(
    classes_dir,
    labels='inferred',
    label_mode='categorical',
    class_names=class_names,
    color_mode='grayscale',
    batch_size=batchSize,
    image_size=imageSize,
    shuffle=True,
    seed=1,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)


Found 942 files belonging to 10 classes.
Using 754 files for training.


In [14]:

gen_data_test = tf.keras.utils.image_dataset_from_directory(
    classes_dir,
    labels='inferred',
    label_mode='categorical',
    class_names=class_names,
    color_mode='grayscale',
    batch_size=batchSize,
    image_size=imageSize,
    shuffle=True,
    seed=1,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)


Found 942 files belonging to 10 classes.
Using 188 files for validation.


## CNN Model 📦

In [17]:
#* CNN model
classifier = tf.keras.Sequential()
classifier.add(Conv2D(filters=16, kernel_size=(4, 4), strides=(1, 1), padding='same', input_shape=(224, 224, 1), activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(filters=16, kernel_size=(4, 4), strides=(1, 1), padding='same', activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())
classifier.add(Dense(16, activation='relu'))
classifier.add(Dense(10, activation='softmax'))


In [18]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.fit(gen_data_train, epochs=20, batch_size=32)

Epoch 1/20
24/24 [==============================] - 17s 670ms/step - loss: 14.1151 - accuracy: 0.0928
Epoch 2/20
24/24 [==============================] - 17s 680ms/step - loss: 2.3032 - accuracy: 0.0902
Epoch 3/20
24/24 [==============================] - 15s 626ms/step - loss: 2.3025 - accuracy: 0.0928
Epoch 4/20
24/24 [==============================] - 15s 597ms/step - loss: 2.3025 - accuracy: 0.0889
Epoch 5/20
24/24 [==============================] - 14s 574ms/step - loss: 2.3025 - accuracy: 0.0862
Epoch 6/20
24/24 [==============================] - 13s 554ms/step - loss: 2.3025 - accuracy: 0.1061
Epoch 7/20
24/24 [==============================] - 17s 706ms/step - loss: 2.3024 - accuracy: 0.1061
Epoch 8/20
24/24 [==============================] - 17s 682ms/step - loss: 2.3024 - accuracy: 0.1061
Epoch 9/20
24/24 [==============================] - 15s 637ms/step - loss: 2.3024 - accuracy: 0.1061
Epoch 10/20
24/24 [==============================] - 16s 647ms/step - loss: 2.3023 - accur

In [19]:
accuracy = classifier.evaluate(gen_data_test,batch_size=32)
print("Accuracy: ",accuracy[1])

6/6 [==============================] - 1s 85ms/step - loss: 2.3047 - accuracy: 0.0904
Accuracy:  0.0904255285859108


In [20]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 16)      272       
                                                                 
 dropout (Dropout)           (None, 224, 224, 16)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 16)      4112      
                                                                 
 dropout_1 (Dropout)         (None, 112, 112, 16)      0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 16)       0         
 2D)                                                    

## change to .h5 to .tflite 📲

In [21]:
# classifier.save("trained_model.h5")

In [22]:
# # Load the trained Keras model from the file
# loaded_model = keras.models.load_model("trained_model.h5")
# # Convert the model to TensorFlow Lite format
# converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
# tflite_model = converter.convert()

# # Save the TFLite model to a file
# with open("trained_model.tflite", "wb") as f:
#     f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\MARIAM~1\AppData\Local\Temp\tmpgk6ykz3k\assets


INFO:tensorflow:Assets written to: C:\Users\MARIAM~1\AppData\Local\Temp\tmpgk6ykz3k\assets
